In [1]:
import os, sys, cv2
import numpy as np
import matplotlib.pyplot as plt

In [3]:
## 사물 분류

filename = '../caffe/fig/coffee-6632524_960_720.webp'

img = cv2.imread(filename)

if img is None:
    print('image read failed')
    sys.exit()
    
model = '../caffe/bvlc_googlenet.caffemodel'
config = '../caffe/deploy.prototxt'

net = cv2.dnn.readNet(model, config)

if net.empty():
    print('network load failed')
    sys.exit()
    
classNames = []

with open('../caffe/classification_classes_ILSVRC2012.txt') as f:
    classNames = f.read().rstrip('\n').split('\n')
    
# print(classNames)
# print(classNames[1])

# blobFromImage(image[, scalefactor[, size[, mean[, swapRB[, crop[, ddepth]]]]]]) -> retval
blob = cv2.dnn.blobFromImage(img, 1, (244, 244), (104, 117, 123), swapRB = False)

net.setInput(blob)
prob = net.forward()
# print(prob)
print(prob.shape)

out = prob.flatten()
# print(out.shape)

ClassId = np.argmax(out)
# print(ClassId)
# print(classNames[ClassId])
# print(out[ClassId])

confidence = out[ClassId]
category = classNames[ClassId]

text = f'{category} ({confidence*100:4.2f} %)'

cv2.putText(img, text, (10, 30), cv2.FONT_HERSHEY_COMPLEX,
           1, (0, 0, 255), 1, cv2.LINE_AA)

cv2.imshow('img', img)

cv2.waitKey()
cv2.destroyAllWindows()

(1, 1000)


In [83]:
## 얼굴 인식

img = cv2.imread('../facedetection/dog-7058195_960_720.webp')

model = '../facedetection/opencv_face_detector_uint8.pb'
config = '../facedetection/opencv_face_detector.pbtxt'                    # 모델의 구조. pbtxt가 tensorflow

# model = '../facedetection/res10_300x300_ssd_iter_140000_fp16.caffemodel'
# config = '../facedetection/deploy.prototxt'

face_detect_net = cv2.dnn.readNet(model, config)

if face_detect_net.empty():
    print('net load failed')
    sys.exit()
    
blob = cv2.dnn.blobFromImage(img, 1, (300, 300), (104, 177, 123), swapRB = False)

face_detect_net.setInput(blob)

out = face_detect_net.forward()               # net을 통과하여 반환하라(두 개의 더미 데이터가 포함되어서 나옴
# print(out.shape) -> (1, 1, 200, 7)

detect = out[0, 0, :, :]
# print(detect.shape) -> (200, 7)

h, w = img.shape[:2]

for i in range(detect.shape[0]):
    confidence = detect[i, 2]
    
    if confidence > 0.5:
        x1 = int(detect[i, 3]*w)
        y1 = int(detect[i, 4]*h)
        x2 = int(detect[i, 5]*w)
        y2 = int(detect[i, 6]*h)
        
        cv2.rectangle(img, (x1, y1), (x2, y2), (0, 0, 255), 2)
        
        text = 'Face : {}%'.format(round(confidence*100, 2))
        
        cv2.putText(img, text, (x1, y1-2), cv2.FONT_HERSHEY_COMPLEX, 0.8, (0, 0, 255), 1, cv2.LINE_AA)

cv2.imshow('image', img)

cv2.waitKey()
cv2.destroyAllWindows()

In [21]:
## 영상에서 얼굴 인식

model = '../facedetection/res10_300x300_ssd_iter_140000_fp16.caffemodel'
config = '../facedetection/deploy.prototxt'

cap = cv2.VideoCapture(0)

if not cap.isOpened():
    print('video open failed')
    sys.exit()
    
net = cv2.dnn.readNet(model, config)

if net.empty():
    print('net load failed')
    sys.exit()
    
while True:
    ret, frame = cap.read()
    
    if not ret:
        print('frame read failed')
        break
        
    blob = cv2.dnn.blobFromImage(frame, 1, (300, 300), (104, 177, 123))
    net.setInput(blob)
    out = net.forward()
    
    detect = out[0, 0, :, :]
    
    h, w = frame.shape[:2]
    
    for i in range(detect.shape[0]):
        confidence = detect[i, 2]
        
        if confidence > 0.5:
            x1 = int(detect[i, 3]*w)
            y1 = int(detect[i, 4]*h)
            x2 = int(detect[i, 5]*w)
            y2 = int(detect[i, 6]*h)
            
            cv2.rectangle(frame, (x1, y1), (x2, y2), (0, 0, 255), 2)
        
            text = 'Face : {}%'.format(round(confidence*100, 2))
        
            cv2.putText(frame, text, (x1, y1-2), cv2.FONT_HERSHEY_COMPLEX, 0.8, (0, 0, 255), 1, cv2.LINE_AA)
    cv2.imshow('frame', frame)
    
    if cv2.waitKey(30) == 27:
        break
        
cap.release()
cv2.destroyAllWindows()

In [27]:
from tensorflow import keras

In [35]:
## 직접 학습하여 활용

(x_train, y_train), (x_test, y_test) = keras.datasets.mnist.load_data()
# print(x_train.shape) -> (60000, 28, 28)
# print(x_train.dtype) -> uint8

x_train = x_train.reshape(60000, 28, 28, 1)/255,
x_test = x_test.reshape(x_test.shape[0], 28, 28, 1)/255.

# print(y_train.shape) -> (60000,)

(60000,)
[5 0 4 1 9 2 1 3 1 4]


In [37]:
model = keras.Sequential()
model.add(keras.layers.Conv2D(32, kernel_size = (3, 3), input_shape = (28, 28, 1),
                             activation = 'relu'))
model.add(keras.layers.Conv2D(64, kernel_size = (3, 3), activation = 'relu'))
model.add(keras.layers.MaxPooling2D(pool_size = 2))
model.add(keras.layers.Flatten())
model.add(keras.layers.Dense(128, activation = 'relu'))
model.add(keras.layers.Dropout(0.3))
model.add(keras.layers.Dense(10, activation = 'softmax'))

model.summary()

Model: "sequential_1"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
conv2d_2 (Conv2D)            (None, 26, 26, 32)        320       
_________________________________________________________________
conv2d_3 (Conv2D)            (None, 24, 24, 64)        18496     
_________________________________________________________________
max_pooling2d_1 (MaxPooling2 (None, 12, 12, 64)        0         
_________________________________________________________________
flatten_1 (Flatten)          (None, 9216)              0         
_________________________________________________________________
dense_2 (Dense)              (None, 128)               1179776   
_________________________________________________________________
dropout_1 (Dropout)          (None, 128)               0         
_________________________________________________________________
dense_3 (Dense)              (None, 10)               

In [38]:
model.compile(loss = 'sparse_categorical_crossentropy',
             optimizer = 'adam',
             metrics = 'accuracy')

In [40]:
modelpath = '../mnist_my_model/{epoch:002d}-{val_loss:.4f}.h5'

checkpoint = keras.callbacks.ModelCheckpoint(filepath = modelpath,
                                       save_best_only = True)

early_stopping = keras.callbacks.EarlyStopping(patience = 10)

history = model.fit(x_train, y_train, epochs = 10, batch_size = 200,
                   verbose = 1, validation_split=0.3,
                   callbacks = [checkpoint, early_stopping])

Epoch 1/10
210/210 [==============================] - 25s 118ms/step - loss: 0.2806 - accuracy: 0.9144 - val_loss: 0.0795 - val_accuracy: 0.9754
Epoch 2/10
210/210 [==============================] - 25s 118ms/step - loss: 0.0771 - accuracy: 0.9772 - val_loss: 0.0578 - val_accuracy: 0.9827
Epoch 3/10
210/210 [==============================] - 25s 121ms/step - loss: 0.0530 - accuracy: 0.9835 - val_loss: 0.0506 - val_accuracy: 0.9841
Epoch 4/10
210/210 [==============================] - 26s 122ms/step - loss: 0.0374 - accuracy: 0.9885 - val_loss: 0.0466 - val_accuracy: 0.9860
Epoch 5/10
210/210 [==============================] - 26s 123ms/step - loss: 0.0297 - accuracy: 0.9903 - val_loss: 0.0491 - val_accuracy: 0.9869
Epoch 6/10
210/210 [==============================] - 25s 121ms/step - loss: 0.0253 - accuracy: 0.9916 - val_loss: 0.0480 - val_accuracy: 0.9863
Epoch 7/10
210/210 [==============================] - 26s 121ms/step - loss: 0.0205 - accuracy: 0.9933 - val_loss: 0.0484 - val_ac

In [41]:
print(model.evaluate(x_test, y_test))

313/313 [==============================] - 2s 5ms/step - loss: 0.0388 - accuracy: 0.9892
[0.03878115862607956, 0.9891999959945679]


In [42]:
model.save('../mnist_onnx/', include_optimizer=False)          # pb 파일 생성

Instructions for updating:
This property should not be used in TensorFlow 2.0, as updates are applied automatically.
Instructions for updating:
This property should not be used in TensorFlow 2.0, as updates are applied automatically.
INFO:tensorflow:Assets written to: ../mnist_onnx/assets


In [43]:
! pip install -U tf2onnx

  Attempting uninstall: flatbuffers
    Found existing installation: flatbuffers 2.0
    Uninstalling flatbuffers-2.0:
      Successfully uninstalled flatbuffers-2.0


ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
tensorflow 2.8.0 requires numpy>=1.20, but you have numpy 1.17.3 which is incompatible.


In [48]:
! python -m tf2onnx.convert --saved-model mnist_onnx --output model_mnist.onnx

C:\Users\YGL\anaconda3\lib\runpy.py:125: RuntimeWarning: 'tf2onnx.convert' found in sys.modules after import of package 'tf2onnx', but prior to execution of 'tf2onnx.convert'; this may result in unpredictable behaviour
  warn(RuntimeWarning(msg))
2022-05-04 16:15:18,170 - WARNING - ***IMPORTANT*** Installed protobuf is not cpp accelerated. Conversion will be extremely slow. See https://github.com/onnx/tensorflow-onnx/issues/1557
2022-05-04 16:15:18,172 - WARNING - '--tag' not specified for saved_model. Using --tag serve
2022-05-04 16:15:18,812 - INFO - Signatures found in model: [serving_default].
2022-05-04 16:15:18,812 - WARNING - '--signature_def' not specified, using first signature: serving_default
2022-05-04 16:15:18,812 - INFO - Output names: ['dense_3']
Instructions for updating:
Use `tf.compat.v1.graph_util.extract_sub_graph`
2022-05-04 16:15:18,947 - WARNING - From C:\Users\YGL\anaconda3\lib\site-packages\tf2onnx\tf_loader.py:706: extract_sub_graph (from tensorflow.python.fra

In [80]:
def on_mouse(event, x, y, flags, param):
    global oldx, oldy
    
    if event == cv2.EVENT_LBUTTONDOWN:
        oldx, oldy = x, y
        
    elif event == cv2.EVENT_MOUSEMOVE:
        if flags & cv2.EVENT_FLAG_LBUTTON:
            cv2.line(img, (oldx, oldy), (x, y), 255, 20, cv2.LINE_AA)
            cv2.imshow('image', img)
            oldx, oldy = x, y
            
def norm_digit(img):
    # 무게 중심 좌표 추출
    m = cv2.moments(img)
    cx = m['m10'] / m['m00']
    cy = m['m01'] / m['m00']
    h, w = img.shape[:2]
    
    # affine 행렬 생성
    aff = np.array([[1, 0, w/2 - (cx + 0.5)], [0, 1, h/2 - (cy + 0.5)]], 
                   dtype=np.float32)
    
    # warpAffine을 이용해 기하학 변환
    dst = cv2.warpAffine(img, aff, (0, 0))
    return dst




net = cv2.dnn.readNet('./model_mnist.onnx')

if net.empty():
    print('net load failed')
    sys.exit()
    
img = np.zeros((400, 400), np.uint8)

cv2.imshow('image', img)

cv2.setMouseCallback('image', on_mouse)

while True:
    key = cv2.waitKey()
    
    if key == 27:
        break
        
    elif key == ord(' '):
        blob = cv2.dnn.blobFromImage(img, 1/255., (28, 28))
        net.setInput(blob)
        prob = net.forward()
        
        _, maxVal, _, maxLoc = cv2.minMaxLoc(prob)
        digit = maxLoc[0]
        
        print(f'{digit} ({maxVal*100:4.2f}%)')
        
        img.fill(0)
        
#         cv2.imshow('image', img)

        
cv2.destroyAllWindows()

3 (100.00%)


--------------

In [20]:
filename = os.listdir('../caffe/fig/')

img_path = []

for i in filename:
    img_name = '../caffe/fig/' + i
    img_path.append(img_name)

if img_path is None:
    print('image read failed')
    sys.exit()

# 딥러닝(이미 완료된) 모델 불러오기    
model = '../caffe/bvlc_googlenet.caffemodel'       # 모델 데이터
config = '../caffe/deploy.prototxt'                # 배치 데이터

net = cv2.dnn.readNet(model, config)              # dnn으로 읽기

if net.empty():
    print('network load failed')
    sys.exit()

# 딥러닝으로 알아낼 수 있는 품목을 리스트로 만들기
classNames = []

with open('../caffe/classification_classes_ILSVRC2012.txt') as f:
    classNames = f.read().rstrip('\n').split('\n')
    

idx = 0

while True:
    
    # img_path는 경로로만 저장되어 있으므로 cv2.imread()로 이미지 읽어내기
    img = cv2.imread(img_path[idx])
    
    # dnn에 훈련된 정보와 동일하게 맞추기(내가 가지고 있는 사진을 변형)
    blob = cv2.dnn.blobFromImage(img, 1, (244, 244), (104, 117, 123), swapRB = False)

    # blob로 맞춘 내용 net(모델, 배치 데이터를 dnn으로 정리한 것)에 넣기
    net.setInput(blob)
    # prob에 net의 자료 저장((1, 1000)의 shape으로 들어가 있음)
    # prob는 각 품목별 확률이 저장되어 있다고 생각하면 편함
    prob = net.forward()
    
    # prob를 평탄화하여 out에 저장
    out = prob.flatten()

    # 평탄화된 자료를 argmax를 이용해 가장 높은 확률을 가지는 품목을 선택(인덱스)
    ClassId = np.argmax(out)

    confidence = out[ClassId]
    category = classNames[ClassId]

    text = f'{category} ({confidence*100:4.2f} %)'

    cv2.putText(img, text, (10, 30), cv2.FONT_HERSHEY_COMPLEX,
               1, (0, 0, 255), 1, cv2.LINE_AA)
    
    cv2.imshow('img', img)
    
    key = cv2.waitKey(1000)
    
    if key == 27:
        break
    elif key == ord('s'):
        key = cv2.waitKey()
        if key == ord('s'):
            key = cv2.waitKey(1000)
        elif key == 27:
            break
        
    
    idx += 1
    
    if idx >= len(img_path):
        idx = 0


cv2.destroyAllWindows()